## Meta-learning experiments 
### Clean(er) code

In [1]:
# Imports

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset, Dataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy.stats as stats
import learn2learn as l2l

# plot layout, darkgrid
sns.set(style="darkgrid")

In [2]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Loading the relevant data
For each experiment, a support set is needed. Here we use four different support sets that contain datasets of average flows and occupancy values (MFD values) calculated based on multiple sampled subsets of 10, 25, 50, and 75 detectors for the cities.  
The query set consist of the average flows and occupancy values (MFD values) calculated based on all available detectors.

In [3]:
# ----- Support DFs
# Support df consisting of datasets from 10 sampled detectors:
support_10 = pd.read_csv('C:/DTU/Speciale/Kode/Speciale/data/processed/reduced loop detectors/10_rand_det_30_times.csv')
support_10 = support_10[~support_10['city'].isin(['Vilnius', 'Munich'])] # Vilnius and Munich are not included in the support set, as they only contain data for 1 day

# Support df consisting of datasets from 25 sampled detectors:
support_25 = pd.read_csv('C:/DTU/Speciale/Kode/Speciale/data/processed/reduced loop detectors/25_rand_det_20_times.csv')
support_25 = support_25[~support_25['city'].isin(['Vilnius', 'Munich'])] # Vilnius and Munich are not included in the support set, as they only contain data for 1 day

# Support df consisting of datasets from 50 sampled detectors:
support_50 = pd.read_csv('C:/DTU/Speciale/Kode/Speciale/data/processed/reduced loop detectors/50_rand_det_20_times.csv')
support_50 = support_50[~support_50['city'].isin(['Vilnius', 'Munich'])]

# Support df consisting of datasets from 75 sampled detectors:
support_75 = pd.read_csv('C:/DTU/Speciale/Kode/Speciale/data/processed/reduced loop detectors/75_rand_det_20_times.csv')
support_75 = support_75[~support_75['city'].isin(['Vilnius', 'Munich'])]
support_cities = support_75['city'].unique()

# ----- Query DF
query_df = pd.read_csv('C:/DTU/Speciale/Kode/Speciale/data/processed/averages.csv')
# capitalize the city names
query_df['city'] = query_df['city'].str.capitalize()
# drop the cities that are not in the support
query_df = query_df[query_df['city'].isin(support_cities)]
query_df['norm_flow'] = query_df.groupby('city')['avg_flow'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
query_df['norm_occ'] = query_df.groupby('city')['avg_occupancy'].transform(lambda x: (x - x.min()) / (x.max() - x.min()))

In [4]:
def denormalize(values, feature_min, feature_max):
    return values * (feature_max - feature_min) + feature_min

Picking from different datasets for each inner step:  

Picking a dataset at random for each taskset (not for each inner step):

In [5]:
class TrafficTaskDataset(Dataset):
    def __init__(self, train_groups, val_groups, k_shots=10, m_shots=5, inner_steps=5):
        """
        Args:
            train_groups (dict): Dictionary of city-wise training data.
            val_groups (dict): Dictionary of city-wise validation data.
            k_shots (int): Number of samples for the support set.
            m_shots (int): Number of samples for the query set.
            inner_steps (int): Number of inner-loop steps (tasks per city).
        """
        self.train_groups = train_groups
        self.val_groups = val_groups
        self.k_shots = k_shots
        self.m_shots = m_shots
        self.inner_steps = inner_steps
        self.cities = list(train_groups.keys())

    def __len__(self):
        return len(self.cities)  # Number of tasks = Number of cities

    def __getitem__(self, idx):
        # Select city for the task
        city = self.cities[idx]
        train_data = self.train_groups[city]
        val_data = self.val_groups[city]

        # Select one dataset N randomly for all inner steps
        selected_N = train_data['N'].sample(1, random_state=43).iloc[0]
        selected_data = train_data[train_data['N'] == selected_N]

        # Sample k_shots * inner_steps for support set
        support_set = selected_data.sample(self.k_shots * self.inner_steps, random_state=43)
        support_occupancy = torch.tensor(support_set['norm_occ'].values, dtype=torch.float32).view(-1, 1)
        support_flow = torch.tensor(support_set['norm_flow'].values, dtype=torch.float32).view(-1, 1)

        # Sample m_shots for query set from validation data
        query_set = val_data.sample(self.m_shots, random_state=43)
        query_occupancy = torch.tensor(query_set['norm_occ'].values, dtype=torch.float32).view(-1, 1)
        query_flow = torch.tensor(query_set['norm_flow'].values, dtype=torch.float32).view(-1, 1)

        return {
            'city': city,
            'support': {'occupancy': support_occupancy, 'flow': support_flow},
            'query': {'occupancy': query_occupancy, 'flow': query_flow},
        }

# function for preparing data for the model
def prepare_data(support_df, query_df, k_shots, m_shots, inner_steps=1):
    # Pick two cities randomly for a test task
    cities = support_df['city'].unique()
    test_cities = np.random.choice(cities, 3, replace=False)

    # Split the data into test and train
    test_support_df = support_df[support_df['city'].isin(test_cities)]
    test_query_df = query_df[query_df['city'].isin(test_cities)]
    support_df = support_df[~support_df['city'].isin(test_cities)]
    query_df = query_df[~query_df['city'].isin(test_cities)]
    
    # Group data by city for easier access
    train_groups = {city: data for city, data in support_df.groupby('city')}
    val_groups = {city: data for city, data in query_df.groupby('city')}
    test_support_groups = {city: data for city, data in test_support_df.groupby('city')}
    test_query_groups = {city: data for city, data in test_query_df.groupby('city')}

    # Instantiating the datasets
    task_dataset = TrafficTaskDataset(train_groups, val_groups, k_shots=k_shots, m_shots=m_shots, inner_steps=inner_steps)
    test_task_dataset = TrafficTaskDataset(test_support_groups, test_query_groups, k_shots=k_shots, m_shots=m_shots, inner_steps=inner_steps)

    # Wrap the task datasets in a DataLoader
    task_loader = DataLoader(task_dataset, batch_size=1, shuffle=True)
    test_task_loader = DataLoader(test_task_dataset, batch_size=1, shuffle=True)

    return task_loader, test_task_loader

### MTPINN model for comparison

The MTPINN model is used for comparison with the meta results

In [6]:
class MultiTaskPINNModel(nn.Module):
    def __init__(self, dropout_prob=0.0):
        super(MultiTaskPINNModel, self).__init__()
        
        # Feature extraction layers
        self.feature_extractor = nn.Sequential(
            nn.Linear(1, 64),
            nn.ReLU(),
            nn.Dropout(p=dropout_prob),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(p=dropout_prob),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(p=dropout_prob)
        )
        
        # Output branch for flow prediction
        self.flow_predictor = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(p=dropout_prob),
            nn.Linear(64, 1)
        )
        
        # Output branch for critical density prediction
        self.cd_predictor = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(p=dropout_prob),
            nn.Linear(64, 1),
            nn.Sigmoid()  # Normalize critical density to be within [0, 1] 
            # We move this to the forward pass to be able to clamp the value to [0.01, 0.99] for stability
        )

        # Output branch for max flow prediction
        self.max_flow_predictor = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(p=dropout_prob),
            nn.Linear(64, 1),
            nn.Sigmoid()  # Normalize max flow to be within [0, 1]
        )

        # Learnable parameter for offset
        self.offset = nn.Parameter(torch.tensor(0.0))  # Initializing offset as a learnable parameter

        # Learnable parameter for occ_scaler (scaling factor for the second parabola)
        self.occ_scaler = nn.Parameter(torch.tensor(3.0))  # Initially set to 3, can be adjusted based on data


    def forward(self, x):
        features = self.feature_extractor(x)

        flow = self.flow_predictor(features)

        critical_density = self.cd_predictor(features) 
        critical_density = torch.clamp(critical_density, 0.05, 0.95) # clamp critical density toi ensure stability by avoiding extreme values
        # critical_density = critical_density.mean(dim=0, keepdim=True)  # Averaging across batch for single prediction value

        max_flow = self.max_flow_predictor(features)
        # max_flow = max_flow.mean(dim=0, keepdim=True)  # Averaging across batch for single value

        # Using the learnable offset and occ_scaler
        offset = self.offset  # Directly use the learned offset parameter
        occ_scaler = self.occ_scaler  # Use occ_scaler for dynamic adjustment of the second parabola's width

        return flow, critical_density, max_flow, offset, occ_scaler

    def compute_total_loss(self, flow, pred_flow, pred_cd, pred_max_flow, pred_offset, pred_occ_scaler, alpha=1.0):
        # Flow MSE loss
        mse_loss_flow = nn.functional.mse_loss(pred_flow, flow)

        # Physics-informed loss (indirectly encourages cd and max flow learning)
        physics_loss = self.physics_informed_loss(flow, pred_flow, pred_cd, pred_max_flow, pred_offset, pred_occ_scaler)

        # Total loss with scaling factor for physics-informed loss
        total_loss = mse_loss_flow + alpha * physics_loss
        return total_loss

    def physics_informed_loss(self, occupancy, flow, critical_density, max_flow, offset, occ_scaler):
        # Define physics-informed loss: encourages the flow to fit a parabolic shape around critical density
        mask_before_cd = occupancy <= critical_density
        mask_after_cd = occupancy > critical_density

        # Use max_flow - offset in parabolic fits
        parabola1_a = -(max_flow - offset) / (critical_density ** 2 + 1e-6)  # Scaling factor for left parabola
        fit1 = parabola1_a * (occupancy - critical_density) ** 2 + (max_flow - offset)

        # parabola2_a = -(max_flow - offset) / ((1 - critical_density) ** 2  + 1e-6)  # Scaling factor for right parabola
        # fit2 = parabola2_a * (occupancy - critical_density) ** 2 + (max_flow - offset)
        # Adjusting the width of the second parabola with occ_scaler
        # parabola2_a = -(max_flow - offset) / ((occ_scaler * (1 - critical_density)) ** 2 + 1e-6)  # Scaling factor for right parabola
        parabola2_a = -(max_flow - offset) / (((occ_scaler - 1) * critical_density) ** 2 + 1e-6)  # Scaling factor for right parabola
        fit2 = parabola2_a * (occupancy - critical_density) ** 2 + (max_flow - offset)

        # MSE for both fits (unless no data points in a segment, then loss is 0)
        mse_fit1 = torch.mean((flow[mask_before_cd] - fit1[mask_before_cd])**2) if torch.sum(mask_before_cd) > 0 else torch.tensor(0.0, device=flow.device)
        mse_fit2 = torch.mean((flow[mask_after_cd] - fit2[mask_after_cd])**2) if torch.sum(mask_after_cd) > 0 else torch.tensor(0.0, device=flow.device)

        # Regularization term for max flow position
        penalty_term = torch.mean(torch.relu(flow - (max_flow)))  # Penalizing flow values higher than (max_flow)

        # Rguarization term for the width of the second parabola, the occ_scaler to be between 1 and 4
        penalty_term2 = torch.mean(torch.relu((occ_scaler - 5 * critical_density))) + torch.mean(torch.relu((2 * critical_density - occ_scaler))) * 5 # here we penalize the occ_scaler to be outside the range [1, 4]

        # Regularization term: Penalize small Fit 2 region
        total_points = torch.sum(mask_before_cd) + torch.sum(mask_after_cd)
        percentage_fit2 = torch.sum(mask_after_cd) / (total_points + 1e-6)  # Avoid division by zero
        threshold = 0.05  # Minimum percentage for Fit 2
        penalty_term3 = torch.relu(threshold - percentage_fit2) * 0.5

        # Encourage cd to align with peak flow region
        peak_flow_penalty = torch.mean((flow - max_flow)**2 * torch.exp(-((occupancy - critical_density)**2) / (2 * 0.05 **2)))

        # Inverse frequency weights, ignoring zero segments
        weight1 = 1 / mask_before_cd.sum() if mask_before_cd.any() else 0.0
        weight2 = 1 / mask_after_cd.sum() if mask_after_cd.any() else 0.0

        # Normalize weights so they sum to 1, provided both segments have data
        total_weight = weight1 + weight2
        if total_weight > 0:
            weight1 /= total_weight
            weight2 /= total_weight

        return weight1 * mse_fit1 + weight2 * mse_fit2 + penalty_term + peak_flow_penalty + penalty_term2 #(width of 2nd parabola + penalty_term3

In [ ]:
# Function for running the MTPINN model as a comparison
def mtpinn_comparison(dropout, 
                      query_df, 
                      save_path, 
                      title_str, 
                      city, 
                      support_occupancy, 
                      support_flow, 
                      query_occupancy, 
                      query_flow, 
                      step):
    
    # Set up path for saving figures:
    save_path = f'{save_path}/mtpinn_compar/'
    os.makedirs(save_path, exist_ok=True)

    # Initialize model
    mtpinn_model = MultiTaskPINNModel(dropout_prob=dropout).to(device)
    mse_loss = nn.MSELoss()

    # Define optimizer and loss
    optimizer = optim.Adam(mtpinn_model.parameters(), lr=0.001)
    alpha = 1.0  # Weight for physics-informed loss
    num_epochs = 100  # Number of training epochs
    batch_size = 5  # Adjust batch size as needed

    # store values
    training_losses = []
    mtpinn_params = {}

    # Supervised training loop
    mtpinn_model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        num_batches = len(support_occupancy) // batch_size
        for batch_idx in range(num_batches):
            # Get batch data
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx + 1) * batch_size
            occupancy_batch = support_occupancy[batch_start:batch_end].to(device)
            flow_batch = support_flow[batch_start:batch_end].to(device)

            # Reset gradients
            optimizer.zero_grad()

            # Forward pass
            pred_flow, pred_cd, pred_max_flow, pred_offset, pred_occ_scaler = mtpinn_model(occupancy_batch)

            # Compute total loss
            loss = mtpinn_model.compute_total_loss(flow_batch, pred_flow, pred_cd, pred_max_flow, pred_offset, pred_occ_scaler, alpha=alpha)

            # Backpropagation and optimization
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        # Store epoch loss
        training_losses.append(epoch_loss / num_batches)

    # Plot training loss
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    ax.plot(training_losses)
    fig.suptitle("Training Loss - " + city)
    ax.set_xlabel("Epoch")
    ax.set_ylabel("PI Total Loss")
    plt.savefig(save_path + f'/{title_str}_mtpinn_loss_{step}_{city}.png')
    plt.show()

    # Evaluate the model on the query set
    mtpinn_model.eval()
    with torch.no_grad():
        query_predictions, query_cd, query_max_flow, query_offset, query_occ_scaler = mtpinn_model(query_occupancy)

        # Compute MSE loss on the query set
        query_loss = mse_loss(query_predictions, query_flow)

        # Calculate RRSE
        true_values = query_flow.cpu().numpy()
        predicted_values = query_predictions.cpu().numpy()
        mean_true_values = np.mean(true_values)
        rrse = np.sqrt(np.sum((predicted_values - true_values) ** 2) / np.sum((true_values - mean_true_values) ** 2))

        # Calculate Correlation Coefficient
        correlation_coefficient = np.corrcoef(true_values.flatten(), predicted_values.flatten())[0, 1]

        # Ensure denormalized values are also NumPy arrays
        query_predictions_denorm = denormalize(query_predictions, query_df['avg_flow'].min(), query_df['avg_flow'].max()).cpu().numpy()
        query_flow_denorm = denormalize(query_flow, query_df['avg_flow'].min(), query_df['avg_flow'].max()).cpu().numpy()

        # Denormalized MSE and RRSE
        query_loss_denorm = np.mean((query_predictions_denorm - query_flow_denorm) ** 2)
        rrse_denorm = np.sqrt(np.sum((query_predictions_denorm - query_flow_denorm) ** 2) / np.sum((query_flow_denorm - query_flow_denorm.mean()) ** 2))
        correlation_coefficient_denorm = np.corrcoef(query_flow_denorm.flatten(), query_predictions_denorm.flatten())[0, 1]

        # Store the parameters
        for key, value in{
            'city': city,
            'cd': query_cd.mean().item(),  # Convert scalar tensor to float
            'max_flow': query_max_flow.max().item(),  # Convert scalar tensor to float
            'offset': query_offset.mean().item(),  # Convert scalar tensor to float
            'occ_scaler': query_occ_scaler.mean().item(),  # Convert scalar tensor to float
            'test_mse': query_loss.item(),  # Convert scalar tensor to float,
            'rrse': rrse,
            'correlation': correlation_coefficient,
            'test_mse_denorm': query_loss_denorm.item(),
            'rrse_denorm': rrse_denorm,
            'correlation_denorm': correlation_coefficient_denorm
        }.items():
            mtpinn_params.setdefault(key, []).append(value)

        # Print the query loss
        print(f"Test Loss: {query_loss.item():.4f}")

        # Plot the query predictions
        # specify linearly spaced occupancy values for plotting the prediction
        occupancy_values = torch.linspace(0, 1, 100).unsqueeze(1)
        line_preds, _, _, _, _ = mtpinn_model(occupancy_values)
        city_data = query_df[query_df['city'] == city]
        
        mse_text = f"MSE: {query_loss.item():.4f}"
        fig, axs = plt.subplots(1, 1, figsize=(8, 5))
        fig.suptitle(f"MTPINN Test Predictions - {city} - {mse_text}")
        axs.plot(city_data['norm_occ'], city_data['norm_flow'], 'o', c='k', alpha=0.5, label="True Flow")
        axs.plot(support_occupancy.view(-1), support_flow.view(-1), 'o', alpha=0.5, label="Support Flow", c='orange')
        axs.plot(occupancy_values, line_preds, label="Predicted Flow Line", color='g', lw=2)
        axs.axvline(query_cd.mean().item(), color='darkred', linestyle=':', lw=2, label="Critical Occupancy: " + str(round(query_cd.mean().item(), 2)))
        axs.axhline(query_max_flow.max().item(), color='darkred', linestyle='--', lw=2, label="Max Flow: " + str(round(query_max_flow.max().item(), 2)))

        axs.set_xlabel("Occupancy", fontsize=16)
        axs.set_ylabel("Flow", fontsize=16)
        axs.legend(fontsize=14)
        plt.savefig(save_path + f'/{title_str}_mtpinn_do_{dropout}_{step}_{city}.png')
        plt.show()

    return mtpinn_params

### The MAML Meta-Learning Model

The code below contains functions for training and testing the meta-model.

In [34]:
# Function that trains the model using MAML with gradient accumulation over multiple tasks
# and returns the trained meta-learner, test task loader, learned parameter history, and title string

def meta_trainer(support_df,
                 query_df,
                 save_path,
                 inner_steps=5,
                 meta_iterations=100,
                 lr_inner=0.01,
                 lr_outer=0.001,
                 k_shots=30,
                 m_shots=100,
                 accumulation_steps=3,
                 step="",
                 dropout=0.0,
                 detectors=75):
    
    # Set up path for saving figures:
    save_path = f'{save_path}/meta_train/'
    os.makedirs(save_path, exist_ok=True)

    # Instantiate base model
    model = MultiTaskPINNModel(dropout_prob=dropout)

    # Prepare data: task loader yields tasks for support & query
    task_loader, test_task_loader = prepare_data(
        support_df, query_df,
        k_shots, m_shots,
        inner_steps=inner_steps)

    # MAML wrapper and optimizers
    maml = l2l.algorithms.MAML(model, lr=lr_inner)
    meta_optimizer = optim.Adam(maml.parameters(), lr=lr_outer)

    # Tracking
    inner_losses = []
    outer_losses = []
    learned_params = {}

    # Meta-training loop with accumulation
    for iteration in range(meta_iterations):
        # Zero meta-gradients at start of accumulation
        meta_optimizer.zero_grad()

        # Accumulate gradients over several tasks
        for acc in range(accumulation_steps):
            # Sample one task
            task = next(iter(task_loader))
            support = task['support']
            query = task['query']

            # Reshape support for inner loop
            support_occupancy = support['occupancy'].view(inner_steps, k_shots, -1)
            support_flow = support['flow'].view(inner_steps, k_shots, -1)

            # Prepare query
            query_occupancy = query['occupancy'].squeeze(0)
            query_flow = query['flow'].squeeze(0)

            # Clone for task-specific adaptation
            learner = maml.clone()

            # Inner-loop adaptation
            for inner_batch in range(inner_steps):
                batch_occ = support_occupancy[inner_batch].requires_grad_()
                batch_flow = support_flow[inner_batch].requires_grad_()

                preds, cd, max_flow, offset, occ_scaler = learner(batch_occ)
                loss = learner.compute_total_loss(
                    batch_flow, preds, cd, max_flow, offset, occ_scaler)
                learner.adapt(loss, allow_unused=True)
                inner_losses.append(loss.item())

            # Evaluate on query
            q_preds, q_cd, q_max_flow, q_offset, q_occ_scaler = learner(query_occupancy)
            q_loss = learner.compute_total_loss(
                query_flow, q_preds, q_cd, q_max_flow, q_offset, q_occ_scaler)
            outer_losses.append(q_loss.item())

            # Record parameter metrics
            metrics = {
                'cd': q_cd.mean().item(),
                'max_flow': q_max_flow.max().item(),
                'offset': q_offset.mean().item(),
                'occ_scaler': q_occ_scaler.mean().item(),
                'parabola1a': -(q_max_flow.mean().item() - q_offset.mean().item())
                              / (q_cd.mean().item()**2 + 1e-6),
                'parabola2a': -(q_max_flow.mean().item() - q_offset.mean().item())
                              / (((q_occ_scaler.mean().item() - 1)*q_cd.mean().item())**2 + 1e-6)
            }
            for key, val in metrics.items():
                learned_params.setdefault(key, []).append(val)

            # Backward for meta-gradient accumulation
            # Optionally normalize by accumulation_steps: q_loss = q_loss / accumulation_steps
            (q_loss / accumulation_steps).backward()

        # Meta-optimization step after accumulating
        meta_optimizer.step()

    # Plotting results
    title_str = f'Meta-learning Experiment - detectors = {detectors} lr_i={lr_inner} lr_o={lr_outer} '
    title_str += f'k={k_shots} m={m_shots} meta_iter={meta_iterations}'

    fig_str = ("Meta-learning Experiment - detectors={}, "
               "alpha={}, beta={}, "
               "k={}, m={}").format(detectors, lr_inner, lr_outer, k_shots, m_shots)
    fig, axs = plt.subplots(2, 1, figsize=(12, 8))
    fig.suptitle(fig_str, fontsize=16, fontweight='bold')
    axs[0].plot(inner_losses, label='Inner Loop Loss')
    axs[0].set_ylabel('Loss', fontsize=14)
    axs[0].set_xlabel('Inner Iterations', fontsize=14)
    axs[0].legend()
    axs[1].plot(outer_losses, label='Meta Loss')
    axs[1].set_ylabel('Loss', fontsize=14)
    axs[1].set_xlabel('Accumulated Tasks', fontsize=14)
    axs[1].legend()

    plt.savefig(f"{save_path}/{title_str}_acc{accumulation_steps}_do_{dropout}_{step}.png")

    return maml, test_task_loader, learned_params, title_str

In [ ]:
# Function for meta-testing the model
def meta_tester(meta_model, 
                query_df, 
                save_path, 
                support_occupancy, 
                support_flow, 
                query_occupancy, 
                query_flow, 
                inner_steps, 
                title_str, 
                city, 
                step, 
                dropout):
    
    # Set up path for saving figures:
    save_path = f'{save_path}/meta_test/'
    os.makedirs(save_path, exist_ok=True)
    
    # Clone model for task-specific adaptation
    learner = meta_model.clone()
    mse_loss = nn.MSELoss()

    meta_test_params = {}
    inner_losses = []

    # Inner-loop updates on support set (fine-tuning)
    for inner_batch in range(inner_steps):
        batch_occupancy = support_occupancy[inner_batch].requires_grad_()
        batch_flow = support_flow[inner_batch].requires_grad_()
        
        support_predictions, pred_cd, pred_max_flow, pred_offset, pred_occ_scaler = learner(batch_occupancy)
        support_loss = learner.compute_total_loss(batch_flow, support_predictions, pred_cd, pred_max_flow, pred_offset, pred_occ_scaler)
        inner_losses.append(support_loss.item())
        learner.adapt(support_loss)
    
    # plot the inner loop losses
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    ax.plot(inner_losses, label="Inner Loop Loss")
    ax.set_ylabel("PI Total Loss", fontsize=16)
    ax.set_xlabel("Iteration", fontsize=16)
    ax.legend(fontsize=14)
    plt.savefig(save_path + f'/{title_str}_inner_loss_do_{dropout}_{step}_{city}.png')
    plt.show()

    # Evaluate on the query set
    with torch.no_grad(): # Disable gradients for query evaluation
        query_predictions, query_cd, query_max_flow, query_offset, query_occ_scaler = learner(query_occupancy)
        
        # Compute MSE loss on the query set
        query_loss = mse_loss(query_predictions, query_flow) # Compute loss on the query set

        # Compute RRSE loss on the query set
        query_flow_mean = query_flow.mean()
        rrse_numerator = torch.sum((query_predictions - query_flow) ** 2)
        rrse_denominator = torch.sum((query_flow - query_flow_mean) ** 2) + 1e-8  # Avoid division by zero
        rrse = torch.sqrt(rrse_numerator / rrse_denominator)

         # Compute correlation coefficient
        query_predictions_mean = query_predictions.mean()
        covariance = torch.sum((query_predictions - query_predictions_mean) * (query_flow - query_flow_mean))
        variance_pred = torch.sum((query_predictions - query_predictions_mean) ** 2)
        variance_true = torch.sum((query_flow - query_flow_mean) ** 2)
        correlation_coefficient = covariance / (torch.sqrt(variance_pred * variance_true) + 1e-8)  # Avoid division by zero

        # Compute the denormalized MSE, RRSE and correlation coefficient
        query_predictions_denorm = denormalize(query_predictions, query_df['avg_flow'].min(), query_df['avg_flow'].max())
        query_flow_denorm = denormalize(query_flow, query_df['avg_flow'].min(), query_df['avg_flow'].max())
        query_loss_denorm = mse_loss(query_predictions_denorm, query_flow_denorm)
        rrse_denorm = torch.sqrt(torch.sum((query_predictions_denorm - query_flow_denorm) ** 2) / torch.sum((query_flow_denorm - query_flow_denorm.mean()) ** 2))
        covariance_denorm = torch.sum((query_predictions_denorm - query_predictions_denorm.mean()) * (query_flow_denorm - query_flow_denorm.mean()))
        variance_pred_denorm = torch.sum((query_predictions_denorm - query_predictions_denorm.mean()) ** 2)
        variance_true_denorm = torch.sum((query_flow_denorm - query_flow_denorm.mean()) ** 2)
        correlation_coefficient_denorm = covariance_denorm / (torch.sqrt(variance_pred_denorm * variance_true_denorm) + 1e-8)  # Avoid division by zero



        for key, value in {
            'city': city,
            'cd': query_cd.mean().item(),
            'max_flow': query_max_flow.max().item(), # changed from mean to max!
            'offset': query_offset.mean().item(),
            'occ_scaler': query_occ_scaler.mean().item(),
            'parabola1a': -(query_max_flow.mean().item() - query_offset.mean().item()) / (query_cd.mean().item() ** 2 + 1e-6),
            'parabola2a': -(query_max_flow.mean().item() - query_offset.mean().item()) / (((query_occ_scaler.mean().item() - 1) * query_cd.mean().item()) ** 2 + 1e-6),
            'mse loss': query_loss.item(),
            'rrse': rrse.item(),
            'correlation': correlation_coefficient.item(),
            'mse loss denorm': query_loss_denorm.item(),
            'rrse denorm': rrse_denorm.item(),
            'correlation denorm': correlation_coefficient_denorm.item()
        }.items():
            meta_test_params.setdefault(key, []).append(value)

        # Plot the query predictions - specify linearly spaced occupancy values for plotting the prediction
        occupancy_values = torch.linspace(0, 1, 100).unsqueeze(1)
        line_preds, _, _, _, _ = learner(occupancy_values)
        city_data = query_df[query_df['city'] == city]
        
        mse_text = f"MSE: {query_loss.item():.4f}"
        fig, axs = plt.subplots(1, 1, figsize=(8, 5))
        fig.suptitle(f"Meta-Test Predictions - {city} - {mse_text}", fontsize=16, fontweight='bold')
        axs.plot(city_data['norm_occ'], city_data['norm_flow'], 'o', c='k', alpha=0.5, label="True Flow")
        axs.plot(support_occupancy.view(-1), support_flow.view(-1), 'o', alpha=0.5, label="Support Set", color='orange')
        axs.plot(occupancy_values, line_preds, label="Predicted Flow Line", color='g', lw=2)
        axs.axvline(query_cd.mean().item(), color='darkred', linestyle=':', lw=2, label="Critical Occupancy: " + str(round(query_cd.mean().item(), 2)))
        axs.axhline(query_max_flow.max().item(), color='darkred', linestyle='--', lw=2, label="Max Flow: " + str(round(query_max_flow.max().item(), 2)))
        
        axs.set_xlabel("Occupancy", fontsize=16)
        axs.set_ylabel("Flow", fontsize=16) 
        axs.legend(fontsize=14)
        plt.savefig(save_path + f'/{title_str}_meta_test_do_{dropout}_{step}_{city}.png')
        plt.show()

    print(f"Meta-Test Query MSE-Loss: {query_loss.item():.4f}")
    
    return meta_test_params 


### Setting up the experiment  

An experiment is training the MAML model once, saving the training losses, the meta-test results and the results from the mtpinn model run for comparison. Each tine the experiment is run, 3 cities are chosen at random and left out of the training data passed to the meta-trainer function. These cities are then used for testing, and the results are stored in CSV files. When running multiple experiments, the test results are stored in the same CSV files.

In [ ]:
# # Setting up a function to "run experiments" i.e. execute the meta-training and testing with different hyperparameters
def run_experiment(support_df, 
                   query_df, 
                   save_path, 
                   num_experiments=1, 
                   inner_steps=5, 
                   meta_iterations=100, 
                   lr_inner=0.01, 
                   lr_outer=0.001, 
                   k_shots=1000, 
                   m_shots=1000, 
                   dropout_maml=0.0, 
                   dropout_mtpinn=0.1, 
                   detectors=75):
    
    # Store results & parameters
    meta_train_params = {}
    meta_test_params = {}
    mtpinn_params = {}

    # Run the experiments
    for n in range(num_experiments):
        # Meta-training
        maml_trained, test_loader, learned_params, title_str = meta_trainer(support_df, query_df, save_path, inner_steps=inner_steps, meta_iterations=meta_iterations, lr_inner=lr_inner, lr_outer=lr_outer, k_shots=k_shots, m_shots=m_shots, step=n, dropout=dropout_maml, detectors=detectors)
  
        for key, value in learned_params.items():
            meta_train_params.setdefault(key, []).append(value)

        # Meta-testing & MTPINN comparison
        for task in test_loader:
            city = task['city'][0]
            support = task['support']
            query = task['query']

            support_occupancy = support['occupancy'].view(inner_steps, k_shots, -1)
            support_flow = support['flow'].view(inner_steps, k_shots, -1)
            query_occupancy = query['occupancy'].squeeze(0)
            query_flow = query['flow'].squeeze(0)
            
            # Meta-test the MAML model
            test_result = meta_tester(
                maml_trained, query_df, save_path, support_occupancy, support_flow, query_occupancy, query_flow, inner_steps, title_str, city, n, dropout_maml
            )
            for key, value in test_result.items():
                meta_test_params.setdefault(key, []).append(value)

            # MTPINN Comparison
            mtpinn_result = mtpinn_comparison(dropout_mtpinn, query_df, save_path, title_str, city, support_occupancy, support_flow, query_occupancy, query_flow, n
            )
            # Update meta_train_data dictionary
            for key, value in mtpinn_result.items():
                mtpinn_params.setdefault(key, []).append(value)

    # Convert single-element lists to scalars
    for key, value in meta_train_params.items():
        if len(value) == 1:
            # print("yep")
            meta_train_params[key] = value[0]

    for key, value in meta_test_params.items():
        meta_test_params[key] = [v[0] if isinstance(v, list) and len(v) == 1 else v for v in value]

    for key, value in mtpinn_params.items():
        mtpinn_params[key] = [v[0] if isinstance(v, list) and len(v) == 1 else v for v in value]

    # Store results in a dataframe
    meta_train_df = pd.DataFrame(meta_train_params)
    meta_test_df = pd.DataFrame(meta_test_params)
    mtpinn_df = pd.DataFrame(mtpinn_params)

    # Ensure column 'city' is a string
    meta_test_df['city'] = meta_test_df['city'].astype(str)
    mtpinn_df['city'] = mtpinn_df['city'].astype(str)

    # Save the results as CSV
    meta_train_df.to_csv(f'{save_path}meta_train_results_{title_str}_.csv', index=False)
    meta_test_df.to_csv(f'{save_path}meta_test_results_{title_str}_.csv', index=False)
    mtpinn_df.to_csv(f'{save_path}mtpinn_results_{title_str}_.csv', index=False)
    return meta_train_df, meta_test_df, mtpinn_df

### Running experiments

In [ ]:
# Running the experiment for 75 detectors, 1000 k_shots and 1000 m_shots
number_experiments = 5      # number of experiments to run, i.e. MAML models to train
inner_steps = 5             # number of inner steps for MAML
meta_iterations = 150       # number of meta-iterations
lr_inner = 0.02             # inner loop learning rate
lr_outer = 0.005            # outer loop learning rate 
k_shots = 150               # number of support samples per task, per inner step
m_shots = 750               # number of query samples per task
dropout_maml = 0.0          # dropout for MAML is always set to 0.0 (as dropout was seen to decrease performance)
dropout_mtpinn = 0.1        # dropout for MTPINN is set to 0.1 (as dropout was seen to increase performance)

base_path = 'C:/DTU/Speciale/Kode/Speciale/meta-learning/test_save_path/'
save_path = f'{base_path}/75_detectors/'

meta_train_params, meta_test_params, mtpinn_params = run_experiment(support_df=support_75, query_df=query_df, save_path=save_path, num_experiments=5, inner_steps=5, meta_iterations=150, lr_inner=0.02, lr_outer=0.005, k_shots=150, m_shots=750, dropout_maml=0.0, dropout_mtpinn=0.1, detectors=75) 

In [ ]:
# Running the experiment for 50 detectors, 1000 k_shots and 1000 m_shots
save_path = f'{base_path}/50_detectors/'

meta_train_params, meta_test_params, mtpinn_params = run_experiment(support_df=support_50, query_df=query_df, save_path=save_path, num_experiments=5, inner_steps=5, meta_iterations=150, lr_inner=0.02, lr_outer=0.005, k_shots=150, m_shots=750, dropout_maml=0.0, dropout_mtpinn=0.1, detectors=50)

In [ ]:
# Running the experiment for 25 detectors, 1000 k_shots and 1000 m_shots
save_path = f'{base_path}/25_detectors/'

meta_train_params, meta_test_params, mtpinn_params = run_experiment(support_df=support_25, query_df=query_df, save_path=save_path, num_experiments=5, inner_steps=5, meta_iterations=150, lr_inner=0.02, lr_outer=0.005, k_shots=150, m_shots=750, dropout_maml=0.0, dropout_mtpinn=0.1, detectors=25)